In [1]:
import os
import pprint
from typing import Dict
import torch
from transformers import AutoModelForCausalLM, MllamaForConditionalGeneration, AutoProcessor

model_id = "meta-llama/Llama-3.2-11B-Vision"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="cpu",
)

processor = AutoProcessor.from_pretrained(model_id)
tokenizer = processor.tokenizer

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

# Prepare finetuning dataset of receipts
It the dataset comes from kaggle `https://www.kaggle.com/datasets/dhiaznaidi/receiptdatasetssd300v2` and contains
- Images under `dataset/images` of receipts
- Extracted data under `dataset/gdt` containing `company`, `total`, `date` and `address`
- Data about extracted data in `data/info_data` contains the data with important words and their coordinates

The expected data should contain `receipt_path`, `schema`(for now just the company, date, total and address), and the `output`

## Load Dataset

In [2]:
from dataclasses import dataclass

@dataclass
class ReceiptData:
    receipt_path: str
    schema: Dict
    output: Dict

In [3]:
import json
from typing import List

fixed_schema = {
    "total": "number//total amount of the invoice",
    "company": "string//the name of the company or person doing the supply",
    "date": "date//the date of the invoice",
    "address": "string//address of the person or company doing the supply",
}

def prepare_dataset() -> List[ReceiptData]:
    dataset: List[ReceiptData] = []
    for receipt in os.listdir("dataset/info_data"):
        with open(f"dataset/info_data/{receipt}", "r") as f:
            receipt_info = json.loads(f.read())
            receipt_id  = receipt_info.get("image_path","").split("/")[5].replace(".jpg","")
            with open(f"dataset/gdt/{receipt_id}.json","r") as df:
                extracted_data = json.loads(df.read())
                for k,v in extracted_data.items():
                    if k in receipt_info.keys():
                        extracted_data[k] = {
                            "value": v,
                            "coordinates": receipt_info[k]
                        }
                    else:
                        extracted_data[k] = {
                            "value": v,
                            "coordinates": {
                                "xmin": 0,
                                "ymin": 0,
                                "xmax": 0,
                                "ymax": 0
                            }
                        }
            receipt_data = ReceiptData(
                receipt_path=receipt_info.get("image_path","").replace("/content/Dataset/train/","dataset/"),
                schema=fixed_schema,
                output=extracted_data
            )
            dataset.append(receipt_data)
    return dataset
data_list = prepare_dataset()
print(len(data_list))
pprint.pprint(data_list[0])

967
ReceiptData(receipt_path='dataset/images/627.jpg',
            schema={'address': 'string//address of the person or company doing '
                               'the supply',
                    'company': 'string//the name of the company or person '
                               'doing the supply',
                    'date': 'date//the date of the invoice',
                    'total': 'number//total amount of the invoice'},
            output={'address': {'coordinates': {'xmax': 0,
                                                'xmin': 0,
                                                'ymax': 0,
                                                'ymin': 0},
                                'value': '12, JALAN TAMPOI 7/4,KAWASAN '
                                         'PERINDUSTRIAN TAMPOI,81200 JOHOR '
                                         'BAHRU,JOHOR'},
                    'company': {'coordinates': {'xmax': 0,
                                                'xmin': 0,


## Tokenize the dataset

In [4]:
from dataclasses import asdict
from datasets import Dataset

def convert_to_dataset(data_list: List[ReceiptData]) -> Dataset:
    data_dict = {
        "prompt": [asdict(record) for record in data_list], 
        "completion": [{"output": record.output} for record in data_list]
    }
    dataset = Dataset.from_dict(data_dict)
    return dataset

dataset = convert_to_dataset(data_list=data_list)
dataset

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 967
})

In [5]:
from typing import Dict
from PIL import Image

def tokenize_batched(examples: Dict[str, List]) -> Dict[str, torch.Tensor]:
    receipt_paths = [item.get("receipt_path") for item in examples["prompt"]]
    schemas = [item.get("schema") for item in examples["prompt"]]
    outputs = [f'{item.get("output")}' for item in examples["completion"]]
    
    images = [[Image.open(path)] for path in receipt_paths] 
    input_prompts = [f'<|image|><|begin_of_text|>{schema}' for schema in schemas]
    
    model_inputs = processor(
        images=images,
        text=input_prompts,
        return_tensors="pt",
        padding="longest",
        truncation=True
    )
      
    completion_tokens = tokenizer(
        outputs,
        return_tensors="pt",
        padding="longest",
        truncation=True
    )

    batch_input_ids = []
    batch_attention_mask = []
    batch_labels = []

    # Determine max length across all combined sequences in this batch
    max_len = 0
    for i in range(len(input_prompts)):
        combined_len = model_inputs["input_ids"][i].shape[0] + completion_tokens["input_ids"][i].shape[0]
        if combined_len > max_len:
            max_len = combined_len
    
    # Optional: Cap max_len to a global maximum for consistency
    # if max_len > 1024: max_len = 1024 

    for i in range(len(input_prompts)):
        prompt_input_ids = model_inputs["input_ids"][i]
        prompt_attention_mask = model_inputs["attention_mask"][i]
        
        output_input_ids = completion_tokens["input_ids"][i]
        output_attention_mask = completion_tokens["attention_mask"][i]

        combined_input_ids = torch.cat((prompt_input_ids, output_input_ids))
        combined_attention_mask = torch.cat((prompt_attention_mask, output_attention_mask))

        labels = combined_input_ids.clone()
        prompt_length = prompt_input_ids.shape[0]
        labels[:prompt_length] = -100

        # Pad to max_len of the batch
        current_len = combined_input_ids.shape[0]
        if current_len < max_len:
            padding_len = max_len - current_len
            combined_input_ids = torch.cat([combined_input_ids, torch.full((padding_len,), tokenizer.pad_token_id, dtype=torch.long)])
            combined_attention_mask = torch.cat([combined_attention_mask, torch.zeros(padding_len, dtype=torch.long)])
            labels = torch.cat([labels, torch.full((padding_len,), -100, dtype=torch.long)])

        batch_input_ids.append(combined_input_ids)
        batch_attention_mask.append(combined_attention_mask)
        batch_labels.append(labels)

    return {
        "input_ids": torch.stack(batch_input_ids),
        "attention_mask": torch.stack(batch_attention_mask),
        "labels": torch.stack(batch_labels),
        "pixel_values": model_inputs["pixel_values"],
        "aspect_ratio_ids": model_inputs["aspect_ratio_ids"],
        "aspect_ratio_mask": model_inputs['aspect_ratio_mask'],
    }

tokenized_dataset = dataset.map(
    tokenize_batched,
    batched=True,
    batch_size=4,
    remove_columns=dataset.column_names
)

Map:   0%|          | 0/967 [00:00<?, ? examples/s]

### Test tokenized dataset

In [6]:
# sample_data = {
#     k: torch.tensor(v, dtype=torch.int32).unsqueeze(0)
#     if k in ["input_ids", "attention_mask", "labels"]
#     else torch.tensor(v).unsqueeze(0)  # for pixel_values and others
#     for k, v in tokenized_dataset[0].items()
# } 
# if 'pixel_values' in sample_data:
#     sample_data['pixel_values'] = sample_data['pixel_values'].to(torch.bfloat16)
# output = model.generate(**sample_data, max_new_tokens=30)
# print(processor.decode(output[0]))

# Prompt Tune the model

In [7]:
finetuned_model_output_dir = "models/llama_finetuned/prompt_tuned"
if not os.path.exists(finetuned_model_output_dir):
    os.makedirs(finetuned_model_output_dir)

In [8]:
from peft import LoraConfig, PromptTuningConfig, PromptTuningInit, TaskType, get_peft_model


prompt_tuning_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.RANDOM,
    num_virtual_tokens=8,
    tokenizer_name_or_path=model_id,
)

finetuned_model = get_peft_model(model, prompt_tuning_config)
finetuned_model.print_trainable_parameters()

trainable params: 32,768 || all params: 9,775,224,848 || trainable%: 0.0003


In [9]:
from tqdm import tqdm
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import DataLoader
from transformers.data.data_collator import default_data_collator

device="cpu"
lr = 3e-2
num_epochs=1

optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
train_dataloader = DataLoader(
    tokenized_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=1, pin_memory=True
)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)
finetuned_model = finetuned_model.to(device)
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = finetuned_model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        break
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=}")

  0%|          | 0/967 [00:00<?, ?it/s]


TypeError: 'NoneType' object is not callable

In [ ]:
# from transformers.data.data_collator import DataCollatorForLanguageModeling
# from transformers.trainer import Trainer
# from transformers.training_args import TrainingArguments

# training_args = TrainingArguments(
#     output_dir=finetuned_model_output_dir, # Where the model predictions and checkpoints will be written
#     use_cpu=True, # This is necessary for CPU clusters. 
#     auto_find_batch_size=True, # Find a suitable batch size that will fit into memory automatically 
#     learning_rate= 3e-2, # Higher learning rate than full fine-tuning
#     num_train_epochs=1 # Number of passes to go through the entire fine-tuning dataset 
# )

# trainer = Trainer(
#     model=finetuned_model,
#     args=training_args,
#     train_dataset=tokenized_dataset,
#     data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
# )

# trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`labels` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
import time

time_now = time.time()
peft_model_path = os.path.join(finetuned_model_output_dir, f"receiptiq_model_{time_now}")
model.save_pretrained(peft_model_path)

# LoRA Finetuning

In [ ]:
lora_finetuned_model_output_dir = "models/llama_finetuned/lora_tuned"
if not os.path.exists(lora_finetuned_model_output_dir):
    os.makedirs(lora_finetuned_model_output_dir)